In [1]:
# Add path
import sys
sys.path.insert(1, '../src')

from BitFlipper import add_BitFlips
from Qubit_Simulator import pad_counts
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
import random
from tqdm import tqdm

def CZ_errors(num_gates):
    qc = QuantumCircuit(2)

    for i in range(num_gates):
        qc.cz(0,1)
    qc.measure_all()

    return qc

In [ ]:
# sim_seeds = np.random.randint(0,1073741824,1048576,dtype=int)
sim_seeds = np.load('../data/sim_seeds.npy')
sim_seeds = (np.array(sim_seeds[0:131072],dtype = int))/10000

array([82688.2679, 10319.8191, 10136.8578, ..., 24547.692 , 74742.158 ,
       42397.1944], shape=(131072,))

In [18]:
round(0.2/(max(e_g1,e_g2)))

57

In [17]:
num_qubits = 2
shots_per_circuit = 2
e_g1 = 0.0035
e_g2 = 0.0027
num_gates = round(0.2/(max(e_g1,e_g2)))
test_circuit = CZ_errors(num_gates)

simulator = AerSimulator()
flipped_counts = np.zeros(4)
bitstring_list = [
        format(i, f'0{num_qubits}b') 
        for i in range(2**num_qubits)
    ]


for sim_seed in tqdm(sim_seeds):
    flipped_circuit = add_BitFlips(test_circuit, seed = sim_seed, gate_errors = [e_g1,e_g2])
    flipped_result = simulator.run(flipped_circuit, shots=shots_per_circuit).result()
    counts = pad_counts(flipped_result.get_counts(),2)
    bitstring_counts = [counts.get(key, 0) for key in bitstring_list]
    flipped_counts += bitstring_counts

print(bitstring_list)
flipped_counts

raw_output = []

for i, bitstr in enumerate(bitstring_list):
    raw_output.extend([bitstr] * int(flipped_counts[i]))

np.savetxt('../data/Filter_data_CZ.csv', np.array(raw_output).reshape(1, -1), delimiter=',', fmt='%s')

# len(raw_output)
# noisy_counts = pad_counts(noisy_result.get_counts(),num_qubits)

100%|██████████| 131072/131072 [03:53<00:00, 560.33it/s]


['00', '01', '10', '11']


In [24]:
new_output = np.array(raw_output)

In [29]:
Readout_57CZQ0 = [s[:-1] for s in new_output]
Readout_57CZQ0
Readout_57CZQ1 = [s[1:] for s in new_output]
Readout_57CZQ1
np.savetxt('../Tutorial/data/Readout_57XQ0.csv', np.array(Readout_57CZQ0).reshape(1, -1), delimiter=',', fmt='%s')
np.savetxt('../Tutorial/data/Readout_57XQ1.csv', np.array(Readout_57CZQ1).reshape(1, -1), delimiter=',', fmt='%s')

In [ ]:
print(float(flipped_counts[3]/sum(flipped_counts)))

P_1X = ( 1 - ( ( 1 - 2 * e_g1) ** num_gates ) ) / 2
P_X1 = ( 1 - ( ( 1 - 2 * e_g2) ** num_gates ) ) / 2

a = ((1+e_g1)**num_gates - (1-e_g1)**num_gates)/2 
b = ((1+e_g2)**num_gates - (1-e_g2)**num_gates)/2
print(a*b)

NameError: name 'flipped_counts' is not defined

In [ ]:
flipped_counts
e_m01 = [0.018,0.012]
e_m10 = [0.0165,0.0145]

full_error_counts = fast_interaction_multiply(flipped_counts,2,e_m01,e_m10)

array([4096.,    0.,    0.,    0.])